In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape(original, do not use)/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
df=pd.read_csv('../../Data/yellow_tripdata_2014-12.csv')

C:\Users\Bro Bro\.conda\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


df.info()

df2=df.head(100)

df2.columns

In [7]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x[' dropoff_longitude']), 
                                             float(x[' dropoff_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 786.4253270626068 seconds ---


In [8]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x[' pickup_longitude']), 
                                             float(x[' pickup_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 786.1270799636841 seconds ---


In [9]:
df.head(3)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_geometry,pickup_geometry
0,VTS,2014-12-12 18:16:00,2014-12-12 18:35:00,3,4.03,-74.014050,40.711707,1,NaN,-73.995627,40.759462,CSH,16.0,1.0,0.5,0.0,0.0,17.5,POINT (-73.995627 40.759462),POINT (-74.01404999999998 40.711707)
1,VTS,2014-12-12 18:18:00,2014-12-12 18:36:00,1,4.10,-73.945877,40.780525,1,NaN,-73.972558,40.740458,CRD,15.0,1.0,0.5,4.0,0.0,20.5,POINT (-73.97255800000001 40.740458),POINT (-73.94587699999998 40.780525)
2,VTS,2014-12-12 18:31:00,2014-12-12 18:35:00,1,0.96,-73.961450,40.796260,1,NaN,-73.955630,40.787760,CSH,5.5,1.0,0.5,0.0,0.0,7.0,POINT (-73.95563 40.78776),POINT (-73.96145 40.79626)


In [10]:
df2_start=df[[' passenger_count',' trip_distance',' fare_amount',' mta_tax',' tip_amount',' tolls_amount','pickup_geometry']]

In [11]:
df2_end=df[['dropoff_geometry']]

In [12]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 104.64689373970032 seconds ---


In [13]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 104.8399019241333 seconds ---


In [14]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1079.3133780956268 seconds ---


In [15]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1061.2890248298645 seconds ---


In [16]:
geo_processing_start

,passenger_count,trip_distance,fare_amount,mta_tax,tip_amount,tolls_amount,pickup_geometry,geometry,index_right,zone,LocationID,borough
0,3,4.03,16.0,0.5,0.0,0.0,POINT (-74.01404999999998 40.711707),POINT (-74.01405 40.71171),260,World Trade Center,261,Manhattan
347,1,2.40,12.0,0.5,2.5,0.0,POINT (-74.013952 40.712705),POINT (-74.01395 40.71270),260,World Trade Center,261,Manhattan
908,1,4.10,19.5,0.5,3.0,0.0,POINT (-74.013969 40.712685),POINT (-74.01397 40.71269),260,World Trade Center,261,Manhattan
935,2,3.50,12.5,0.5,2.7,0.0,POINT (-74.01191699999998 40.709341),POINT (-74.01192 40.70934),260,World Trade Center,261,Manhattan
1211,1,1.30,6.0,0.5,1.5,0.0,POINT (-74.01313299999998 40.708358),POINT (-74.01313 40.70836),260,World Trade Center,261,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...
12267331,1,0.19,3.0,0.5,0.0,0.0,POINT (-74.12337999999998 40.5624),POINT (-74.12338 40.56240),175,Oakwood,176,Staten Island
9416867,1,0.15,100.0,0.0,0.0,0.0,POINT (-74.239377 40.521917),POINT (-74.23938 40.52192),43,Charleston/Tottenville,44,Staten Island
9620658,5,3.68,13.0,0.5,2.0,0.0,POINT (-74.103385 40.620207),POINT (-74.10339 40.62021),244,West Brighton,245,Staten Island
10213746,1,0.00,60.0,0.0,0.0,0.0,POINT (-74.08791999999998 40.6331),POINT (-74.08792 40.63310),244,West Brighton,245,Staten Island


In [17]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [18]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [19]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [20]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [21]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [24]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2014-12-processed.csv')
complete

In [23]:
print("--- %s seconds ---" % (time.time() - full_time))

--- 3968.8464663028717 seconds ---
